In [1]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
import warnings
warnings.filterwarnings('ignore')

In [ ]:
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'genres/{g}'):
        songname = f'genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

In [2]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [ ]:
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'./MIR/genres/{g}'):
        songname = f'./MIR/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [3]:
data = pd.read_csv('data.csv')

In [4]:
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00000.au,0.349943,0.130225,1784.420446,2002.650192,3806.485316,0.083066,-113.596742,121.557302,-19.158825,...,8.810668,-3.667367,5.751690,-5.162761,0.750947,-1.691937,-0.409954,-2.300208,1.219928,blues
1,blues.00001.au,0.340983,0.095918,1529.835316,2038.617579,3548.820207,0.056044,-207.556796,124.006717,8.930562,...,5.376802,-2.239119,4.216963,-6.012273,0.936109,-0.716537,0.293875,-0.287431,0.531573,blues
2,blues.00002.au,0.363603,0.175573,1552.481958,1747.165985,3040.514948,0.076301,-90.754394,140.459907,-29.109965,...,5.789265,-8.905224,-1.083720,-9.218359,2.455805,-7.726901,-1.815724,-3.433434,-2.226821,blues
3,blues.00003.au,0.404779,0.141191,1070.119953,1596.333948,2185.028454,0.033309,-199.431144,150.099218,5.647594,...,6.087676,-2.476420,-1.073890,-2.874777,0.780976,-3.316932,0.637981,-0.619690,-3.408233,blues
4,blues.00004.au,0.308590,0.091563,1835.494603,1748.362448,3580.945013,0.101500,-160.266031,126.198800,-35.605448,...,-2.806385,-6.934122,-7.558619,-9.173552,-4.512166,-5.453538,-0.924162,-4.409333,-11.703781,blues


In [5]:
data = data.drop(['filename'],axis=1)

## Encoding Labels

In [6]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [7]:
genre_list = data.iloc[:,-1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [8]:
#Scaling the feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1],dtype = float))

## Train Test Split

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [10]:
len(X_train)

800

In [11]:
X_train[10]

array([ 0.20414156, -0.21833446, -0.59198934, -0.13033398, -0.47654085,
       -1.02162458, -0.07265874,  0.17783521,  0.12677142,  0.69989292,
        1.02246769,  0.03857519,  1.68735339, -0.33537638,  1.66870645,
       -0.67140849,  0.73236034, -0.69854076,  0.67379101, -1.16092723,
        0.44096308, -0.39685095, -0.66228564, -0.75926104,  0.30637607,
       -0.70824748])

## Model

In [12]:
import keras

Using TensorFlow backend.


In [14]:
from keras import models
from keras import layers

In [15]:
model = models.Sequential()
model.add(layers.Dense(256,activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

In [16]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [17]:
history = model.fit(X_train, y_train, epochs=20, batch_size=128)

Epoch 1/20
800/800 [==============================] - 0s 436us/step - loss: 2.1514 - acc: 0.2387
Epoch 2/20
800/800 [==============================] - 0s 22us/step - loss: 1.8374 - acc: 0.3337
Epoch 3/20
800/800 [==============================] - 0s 19us/step - loss: 1.6317 - acc: 0.4238
Epoch 4/20
800/800 [==============================] - 0s 20us/step - loss: 1.4642 - acc: 0.4925
Epoch 5/20
800/800 [==============================] - 0s 20us/step - loss: 1.3335 - acc: 0.5250
Epoch 6/20
800/800 [==============================] - 0s 19us/step - loss: 1.2186 - acc: 0.5925
Epoch 7/20
800/800 [==============================] - 0s 20us/step - loss: 1.1294 - acc: 0.6187
Epoch 8/20
800/800 [==============================] - 0s 22us/step - loss: 1.0442 - acc: 0.6600
Epoch 9/20
800/800 [==============================] - 0s 20us/step - loss: 0.9829 - acc: 0.6763
Epoch 10/20
800/800 [==============================] - 0s 19us/step - loss: 0.9177 - acc: 0.6950
Epoch 11/20
800/800 [=================

In [18]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

200/200 [==============================] - 0s 189us/step


In [19]:
print('test_accuracy',test_accuracy)

test_accuracy 0.665


In [20]:
#Validation
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

In [21]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Train on 600 samples, validate on 200 samples
Epoch 1/30
600/600 [==============================] - 0s 467us/step - loss: 2.2935 - acc: 0.1383 - val_loss: 2.1827 - val_acc: 0.2850
Epoch 2/30
600/600 [==============================] - 0s 32us/step - loss: 2.1090 - acc: 0.3783 - val_loss: 2.0760 - val_acc: 0.3000
Epoch 3/30
600/600 [==============================] - 0s 35us/step - loss: 1.9550 - acc: 0.3850 - val_loss: 1.9777 - val_acc: 0.3150
Epoch 4/30
600/600 [==============================] - 0s 32us/step - loss: 1.8170 - acc: 0.3950 - val_loss: 1.8691 - val_acc: 0.3300
Epoch 5/30
600/600 [==============================] - 0s 32us/step - loss: 1.6782 - acc: 0.4333 - val_loss: 1.7406 - val_acc: 0.3500
Epoch 6/30
600/600 [==============================] - 0s 35us/step - loss: 1.5508 - acc: 0.4800 - val_loss: 1.6278 - val_acc: 0.4050
Epoch 7/30
600/600 [==============================] - 0s 33us/step - loss: 1.4430 - acc: 0.5300 - val_loss: 1.5342 - val_acc: 0.4500
Epoch 8/30
600/600 [==

In [22]:
results

[1.0787119102478027, 0.64]

In [23]:
#Predictions
predictions = model.predict(X_test)

In [24]:
predictions[0].shape

(10,)

In [25]:
np.sum(predictions[0])

0.9999999